In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2
import pickle
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm import tnrange
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

#Check GPU support, please do activate GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def extract_sample(n_way, n_support, n_query, datax, datay):
  """
  Picks random sample of size n_support+n_querry, for n_way classes
  Args:
      n_way (int): number of classes in a classification task
      n_support (int): number of labeled examples per class in the support set
      n_query (int): number of labeled examples per class in the query set
      datax (np.array): dataset of images
      datay (np.array): dataset of labels
  Returns:
      (dict) of:
        (torch.Tensor): sample of images. Size (n_way, n_support+n_query, (dim))
        (int): n_way
        (int): n_support
        (int): n_query
  """
  sample = []
  #extract random classes
  K = np.random.choice(np.unique(datay), n_way, replace=False)
  #extract samples from each claass
  for cls in K:
    datax_cls = datax[datay == cls]
    perm = np.random.permutation(datax_cls)
    sample_cls = perm[:(n_support+n_query)]
    sample.append(sample_cls)
  sample = np.array(sample)
  sample = torch.from_numpy(sample).float()
  sample = sample.permute(0,1,4,2,3)
  return({
      'images': sample,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query
      })
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
    """
    Loads the prototypical network model
    Arg:
        x_dim (tuple): dimension of input image
        hid_dim (int): dimension of hidden layers in conv blocks
        z_dim (int): dimension of embedded image
    Returns:
        Model (Class ProtoNet)
    """
    x_dim = kwargs['x_dim']
    z_dim = kwargs['z_dim']

    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    encoder = nn.Sequential(
        conv_block(x_dim[0], 64),   # -> (64, 42, 42)
        conv_block(64, 64),         # -> (96, 21, 21)
        conv_block(64, 64),        # -> (128, 11, 11)
        conv_block(64, z_dim)        # -> (256, 6, 6)
    )

    # Initialize weights using Kaiming Normal initialization
    for m in encoder.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):  # If there are linear layers
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    return ProtoNet(encoder)
class ProtoNet(nn.Module):
  def __init__(self, encoder):
    """
    Args:
        encoder : CNN encoding the images in sample
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
    """
    super(ProtoNet, self).__init__()
    self.encoder = encoder.to(device)

  def set_forward_loss(self, sample):
    """
    Computes loss, accuracy and output for classification task
    Args:
        sample (torch.Tensor): shape (n_way, n_support+n_query, (dim))
    Returns:
        torch.Tensor: shape(2), loss, accuracy and y_hat
    """
    sample_images = sample['images'].to(device)
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']

    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]

    #target indices are 0 ... n_way-1
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.to(device)

    #encode images of the support and the query set
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)

    z = self.encoder.forward(x)
    z_dim = z.size(-1) #usually 64
    # Extract support and query samples
    z_samples = z[:n_way * n_support]
    z_query = z[n_way * n_support:]

    # Ensure gradients are tracked
    z_samples.requires_grad_(True)
    z_query.requires_grad_(True)
    #----------------------------Making of prototypes---------------------------
    z_proto = Concept_Prototype(z_samples,n_way,n_support)
    #-----------------calculationg distance from each prototypes----------------
    dists = euclidean_dist(z_query,z_proto)
    #--------------compute probabilities,loss and accuracy----------------------
    log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()

    # Flatten predictions and targets
    y_true = target_inds.squeeze().reshape(-1).cpu().numpy()
    y_pred = y_hat.reshape(-1).cpu().numpy()

    # Compute macro F1
    f1_val = f1_score(y_true, y_pred, average='macro')


    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hat,
        'f1_val': f1_val
        }
#function for forming the concept prototypes
# ---------------------------- Channel Statistics ---------------------------- #

def Concept_Prototype(z_samples, n_way, n_support):
    Prototypes = []

    for i in range(n_way):
        class_samples = z_samples[i * n_support:(i + 1) * n_support]
        Class_wise_prototypes = []

        for j in class_samples:  # j: [C, H, W]
            C, H, W = j.shape
            flat_j = j.view(C, -1)   # [C, H*W]

            # --------- Compute Entropy per Channel ---------
            probs = torch.softmax(flat_j, dim=1)  # [C, H*W]
            entropy = -(probs * torch.log(probs + 1e-12)).sum(dim=1)  # [C]

            # --------- Normalize Entropy (per sample) ---------
            entropy_norm = (entropy - entropy.min()) / (entropy.max() - entropy.min() + 1e-12)  # [C]

            # --------- Weight Feature Map by Normalized Entropy ---------
            weighted_feature_map = j * entropy_norm.view(C, 1, 1)  # scale each channel

            Class_wise_prototypes.append(weighted_feature_map)

        # --------- Average Across Support Samples ---------
        Class_wise_prototypes = torch.stack(Class_wise_prototypes)  # [n_support, C, H, W]
        Class_wise_prototypes = torch.mean(Class_wise_prototypes, dim=0)  # [C, H, W]
        Prototypes.append(Class_wise_prototypes)

    # --------- Stack All Class Prototypes ---------
    Prototypes = torch.stack(Prototypes)  # [n_way, C, H, W]
    return Prototypes

#------------------calculation of euclidean distance---------------------------
def euclidean_dist(x, y):
    """
    Computes euclidean distance between x and y
    Args:
        x (torch.Tensor): shape (n, c, h, w). n usually n_query, c is the number of channels, h is height, w is width.
        y (torch.Tensor): shape (m, c, h, w). m usually n_way, c is the number of channels, h is height, w is width.
    Returns:
        torch.Tensor: shape(n, m). For each query, the distances to each centroid
    """
    n = x.size(0)
    m = y.size(0)
    d = x.size(1) * x.size(2) * x.size(3)  # Flattened dimension
    assert x.size(1) == y.size(1) and x.size(2) == y.size(2) and x.size(3) == y.size(3)

    x = x.view(n, 1, d).expand(n, m, d)
    y = y.view(1, m, d).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size):
    """
    Trains the protonet
    Args:
        model
        optimizer
        train_x (np.array): images of training set
        train_y(np.array): labels of training set
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
        max_epoch (int): max epochs to train on
        epoch_size (int): episodes per epoch
    """
    # Divide the learning rate by 2 at each epoch, as suggested in the paper
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    epoch = 0  # Epochs done so far
    stop = False  # Status to know when to stop

    # Lists to store loss and accuracy for plotting
    epoch_losses = []
    epoch_accuracies = []

    while epoch < max_epoch and not stop:
        running_loss = 0.0
        running_acc = 0.0

        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):
            sample = extract_sample(n_way, n_support, n_query, train_x, train_y)
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample)
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / epoch_size
        epoch_acc = running_acc / epoch_size
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch + 1, epoch_loss, epoch_acc))

        # Store the values for plotting
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(epoch_acc)

        epoch += 1
        scheduler.step()

    # Call the plotting function after training
    #plot_training_progress(epoch_losses, epoch_accuracies)

def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
    """
    Tests the protonet
    Args:
        model: trained model
        test_x (np.array): images of testing set
        test_y (np.array): labels of testing set
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
        test_episode (int): number of episodes to test on
    """
    model.eval()

    loss_list = []
    acc_list = []
    f1_list = []

    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    for episode in range(test_episode):
        sample = extract_sample(n_way, n_support, n_query, test_x, test_y)
        loss, output = model.set_forward_loss(sample)

        running_loss += output['loss']
        running_acc += output['acc']
        running_f1 += output['f1_val']

        # Append the current loss and accuracy for later plotting
        loss_list.append(output['loss'])
        acc_list.append(output['acc'])
        f1_list.append(output['f1_val'])

    avg_loss = running_loss / test_episode
    avg_acc = running_acc / test_episode
    avg_f1 = running_f1 / test_episode
    print('Test results -- Loss: {:.4f} Acc: {:.4f} F1-Score: {:.4f}'.format(avg_loss, avg_acc, avg_f1))
    return avg_acc,avg_f1



#-----------Main function-----------------------------------------------------

n_way = 2
n_support = 1
n_query = 3
epochs_to_run = [20,25,30,35,40]
epoch_size = 2000
test_episode = 600
#-------------------------set seeeds-----------------------------
np_seed = 0 # Set the seed for reproducibility
torch_seed = 0
Size=224
# Fix all seeds
np.random.seed(np_seed)
torch.manual_seed(torch_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(torch_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load pickle files
PATH = '/content/gdrive/MyDrive/Refined-Pickle-Files/Refined-Pickle-Files/Pickle-'+str(Size)+'_'+str(Size)+'/'
Data='Derm7pt'

with open(PATH + Data + "_trainx.pkl", "rb") as f:
    trainx = pickle.load(f)
with open(PATH + Data + "_trainy.pkl", "rb") as f:
    trainy = pickle.load(f)
with open(PATH + Data + "_testx.pkl", "rb") as f:
    testx = pickle.load(f)
with open(PATH + Data + "_testy.pkl", "rb") as f:
    testy = pickle.load(f)

# Function to track the highest accuracy
def get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode):
    # Assuming this function is available in your framework
    return test(model, testx, testy, n_way, n_support, n_query, test_episode)

best_accuracy = 0
best_f1 = 0
best_epoch = 0

# Training and testing for specified epochs
for epoch in epochs_to_run:
    # Initialize model and optimizer
    model = load_protonet_conv(x_dim=(3, Size, Size), z_dim=256)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"-----------------------Training for {epoch} epochs-----------------------")
    train(model, optimizer, trainx, trainy, n_way, n_support, n_query, epoch, epoch_size)

    print(f"-----------------------Testing after {epoch} epochs-----------------------")
    accuracy, f1_val = get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode)

    print(f"Accuracy after {epoch} epochs: {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch

print(f"\n Highest accuracy: {best_accuracy:.4f} achieved at epoch: {best_epoch}")



-----------------------Training for 20 epochs-----------------------


/tmp/ipython-input-3150648555.py:256: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


Epoch 1 train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 1 -- Loss: 207.5227 Acc: 0.5988


/tmp/ipython-input-3150648555.py:256: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


...

/tmp/ipython-input-3150648555.py:256: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


Epoch 18 train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 18 -- Loss: 0.8229 Acc: 0.6451


/tmp/ipython-input-3150648555.py:256: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):


Epoch 19 train:   0%|          | 0/2000 [00:00<?, ?it/s]